In [1]:
#24+严绮翘+191203101698
import re
import os
from pyecharts .globals import ChartType, SymbolType
from pyecharts import options as opts
from pyecharts.charts import Grid, Liquid, Funnel, Page, Bar, Pie, Map, Geo, Line
from pyecharts.faker import Collector
import matplotlib.pyplot as plt
import pandas as pd
from wordcloud import WordCloud
from pyecharts.faker import Collector, Faker
import time
from datetime import time
from pyecharts.commons.utils import JsCode
import warnings
import numpy as np
import seaborn as sns
%matplotlib inline
# sns.set()
# import matplotlib
# matplotlib.use('qt5agg')
#指定默认字体
plt.rcParams['font.sans-serif'] = ['SimHei']
# matplotlib.rcParams['font.family']='sans-serif'
#解决负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

In [20]:
df = pd.read_csv('A.csv', encoding='GBK')

In [21]:
df['MaxHR'].max()

202

In [22]:
df['MaxHR'].min()

60

In [23]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [25]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


Cholesterol有从295行开始有连续123行的数据值为0的情况，因为血清胆固醇为0的情况非常少见，由此合理推测这些数据为缺失值，为了不影响后续对Cholesterol与其他因素的综合分析，权衡之下我们选择平均值填充Cholesterol为0的数据，这也是为了任务2中讨论RestingBP 和 Cholesterol 的大小关系不受影响

In [26]:
# from sklearn.utils import shuffle

mm = df[df['HeartDisease']==0]
mm.loc[mm['Cholesterol']==0,'Cholesterol']=mm['Cholesterol'].mean()
nn = df[df['HeartDisease']==1]
nn.loc[nn['Cholesterol']==0,'Cholesterol']=nn['Cholesterol'].mean()
print('mm',mm['Cholesterol'].mean())
print('nn',nn['Cholesterol'].mean())
cc = pd.concat([mm,nn])
cc= cc.reindex(index=range(len(cc)))
cc.shape
df = cc

mm 238.20107079119558
nn 228.58469216938485


C:\anaconda\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\anaconda\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [27]:
df.loc[449,'RestingBP'] = nn['RestingBP'].mean()
df.loc[449]

Age                    55
Sex                     M
ChestPainType         NAP
RestingBP         134.185
Cholesterol       175.941
FastingBS               0
RestingECG         Normal
MaxHR                 155
ExerciseAngina          N
Oldpeak               1.5
ST_Slope             Flat
HeartDisease            1
Name: 449, dtype: object

通过用info()函数观察数据是否存在空值，可以发现918行数据里不存在空值，不用进行缺失值处理

将 CSV 文件的数据先按照 Age 进行排序，然后取出前 100 条数据(降序)，并命名为“task1_1.csv”保存在结果文件夹中。

In [28]:
df1 = df.sort_values(by='Age',ascending=False)#降序
df1.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
814,77,M,ASY,125.0,304.0,0,LVH,162,Y,0.0,Up,1
447,77,M,ASY,124.0,171.0,0,ST,110,Y,2.0,Up,1
688,76,F,NAP,140.0,197.0,0,ST,116,N,1.1,Flat,0
541,76,M,NAP,104.0,113.0,0,LVH,120,N,3.5,Down,1
556,75,M,ASY,160.0,310.0,1,Normal,112,Y,2.0,Down,0


In [29]:
df1 = df1[:100]

In [30]:
task1 = 'task1.csv'
if os.path.exists(task1):
    os.remove(task1)
df1.to_csv('task1.csv', mode='a+', encoding='GBK', header=True, index=0)

In [31]:
df2 = pd.read_csv('task1.csv', encoding='GBK')
data = df2[df2['HeartDisease']==1]
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,77,M,ASY,125.0,304.0,0,LVH,162,Y,0.0,Up,1
1,77,M,ASY,124.0,171.0,0,ST,110,Y,2.0,Up,1
3,76,M,NAP,104.0,113.0,0,LVH,120,N,3.5,Down,1
5,75,M,ASY,136.0,225.0,0,Normal,112,Y,3.0,Flat,1
6,75,M,ASY,170.0,203.0,1,ST,108,N,0.0,Flat,1


In [32]:
data1 = data.groupby('Age').agg({'HeartDisease': 'count'}).sort_values('HeartDisease', ascending=False)
data1

,HeartDisease
Age,
65,12
67,12
69,10
66,7
68,6
70,6
74,5
72,3
71,2


In [33]:
# a = data1['HeartDisease'].value_counts()[:]
da_pair = [list(z) for z in zip(data1.index, data1['HeartDisease'])]
print(da_pair)
(
    #初始化配置项，内部可设置颜色
    Pie(init_opts=opts.InitOpts(bg_color="#fff"))
    .add(
        #系列名称，即该饼图的名称
        series_name="年龄对应HeartDisease为1的条数",
        #系列数据项，格式为[(key1,value1),(key2,value2)]
        data_pair=da_pair,
        #通过半径区分数据大小 “radius” 和 “area” 两种
        rosetype="radius",
        #饼图的半径，设置成默认百分比，相对于容器高宽中较小的一项的一半
        radius="55%",
        #饼图的圆心，第一项是相对于容器的宽度，第二项是相对于容器的高度
        center=["50%", "50%"],
        #标签配置项
        label_opts=opts.LabelOpts(is_show=False, position="center"),
        
    )
    #全局设置
    .set_global_opts(
        #设置标题
        title_opts=opts.TitleOpts(
            #名字
            title="年龄对应HeartDisease为1的条数",
            #组件距离容器左侧的位置
            pos_left="center",
            #组件距离容器上方的像素值
            pos_top="20",
            #设置标题颜色
            title_textstyle_opts=opts.TextStyleOpts(color="#2c343c"),
        ),
        #图例配置项，参数 是否显示图里组件
        legend_opts=opts.LegendOpts(is_show=False),
    )
    #系列设置
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        #设置标签颜色
        label_opts=opts.LabelOpts(color="rgba(0, 0, 0, 0.5)"),
    )
    .render("年龄对应HeartDisease为1的条数.html")
)


[[65, 12], [67, 12], [69, 10], [66, 7], [68, 6], [70, 6], [74, 5], [72, 3], [71, 2], [75, 2], [77, 2], [73, 1], [76, 1]]


'C:\\Users\\寒涧松\\Desktop\\新建文件夹\\年龄对应HeartDisease为1的条数.html'

In [34]:
data2 = df[df['HeartDisease']==1]
counts = data2.HeartDisease.sum()
B = data2['ChestPainType'].value_counts()
B

ASY    392
NAP     72
ATA     24
TA      20
Name: ChestPainType, dtype: int64

In [35]:
R_ASY = B[0]/counts
R_NAP = B[1]/counts
R_ATA = B[2]/counts
R_TA = B[3]/counts
# print("R_ASY,R_NAP,R_ATA,R_TA的值分别是",R_ASY,R_NAP,R_ATA,R_TA)
print("ASY,NAP,ATA,TA的百分比占比分别是{:.2%},{:.2%},{:.2%},{:.2%}".format(R_ASY,R_NAP,R_ATA,R_TA))

ASY,NAP,ATA,TA的百分比占比分别是77.17%,14.17%,4.72%,3.94%


In [36]:
df.to_csv('df2.csv', mode='a+', encoding='GBK', header=True, index=0)